In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


    # 平均值  
    # P1
    # P2     
    # P1-P2  
    # P1/P2  
    # D12
    # D21
    # D11
    # D21/D12
    # Skew1
    # Skew2  
    # Kurt1
    # Kurt2  
    # 方差
    # PeakToPeak
    # Zero Crossing Rate  
    # Energy 
    # 10s内有多少个周期   

# 载入数据，特征矩阵 X 和目标向量 y
X = np.load("./data/train_new_features_norm.npy")
# X = np.load("./data/features_train_without_resp_norm.npy")
X = X + 1
X = np.delete(X, -10, axis=1)
y = np.load("./data/train_without_resp.npy")[:,1004]

print(X.shape)

(3879, 66)


In [32]:
# 定义基因操作 0不动 1乘 2除 46
operations = np.array([0,1,2])

# 定义遗传算法参数
population_size = 500
num_generations = 1000
mutation_rate = 0.5

def evaluate_fitness(individual):
    # 根据操作序列 individual 构建特征组合
    feature_combination = build_feature_combination(X, individual)
    
    # 训练模型并计算适应度（例如，使用线性回归模型）
    model = LinearRegression()
    model.fit(feature_combination, y)
    predictions = model.predict(feature_combination)
    fitness = np.mean(np.abs(predictions - y))  # 均方误差作为适应度
    
    return fitness

def build_feature_combination(X, individual):
    # 根据操作序列 individual 构建特征组合
    feature_combination = [[]]
    flag = 0
    for i in range(len(individual)):
        if individual[i] == 0:
            continue
        elif individual[i] == 1:
            if flag == 0:
                feature_combination = X[:,i]
                flag = 1
            else:
                feature_combination = feature_combination * X[:,i]
        elif individual[i] == 2:
            if flag == 0:
                feature_combination = X[:,i]
                flag = 1
            else:
                feature_combination = feature_combination / X[:,i]
    if flag == 0:
        return X[:,1].reshape(-1,1)
    return np.array(feature_combination).reshape(-1,1)

def genetic_algorithm():
    # 初始化种群
#     population = np.random.randint(0, 3, size=(population_size, len(X[0])))
    population = np.zeros((population_size, len(X[0])))
#     population[:,1] = 1

    best_individual = None  # 用于存储当前最优个体
    best_fitness = float('inf')
    
    for generation in range(num_generations):
        # 计算适应度
        fitness_scores = [evaluate_fitness(individual) for individual in population]
      
    
        # 精英保留
        elite_size = int(0.1 * population_size)
        elite_individuals = np.argsort(fitness_scores)[:elite_size]
        
        # 选择父代个体
        parents = np.argsort(fitness_scores)[:population_size // 5]
        
        # 创建子代个体
        children = []
        for i in range(population_size - elite_size):
            parent1 = population[np.random.choice(parents)]
            parent2 = population[np.random.choice(parents)]
            child = np.where(np.random.rand(len(X[0])) < 0.5, parent1, parent2)
            
            # 变异操作
            if np.random.rand() < mutation_rate:
                mutated_gene = np.random.randint(len(operations))
                mutation_position = np.random.randint(len(X[0]))
                child[mutation_position] = mutated_gene
            
            children.append(child)
        
        next_generation = np.concatenate((population[elite_individuals], np.array(children)))
        population = next_generation
    
    # 找到最优个体
    best_individual = population[np.argmin(fitness_scores)]
    
    return best_individual

best_individual = genetic_algorithm()

# 使用最优个体生成特征组合并进行预测
best_feature_combination = build_feature_combination(X, best_individual)
model = LinearRegression()
model.fit(best_feature_combination, y)
predictions = model.predict(best_feature_combination)

print(best_individual)

[0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 1.]


In [33]:
print(evaluate_fitness(best_individual))
# tmp = np.load("./data/features_train_without_resp_norm.npy")[:,-10].reshape(-1,1)
# model2 = LinearRegression()
# model2.fit(tmp, y)
# predictions = model2.predict(tmp)
# fitness = np.mean(np.abs(predictions - y))  # 均方误差作为适应度
# print(fitness)

7.736125986542999
